Ici on récupère les données via PISTE l'API permettant d'avoir accès aux données de Légifrance, le code de connexion OAuth est donné sur le Gitlab de Piste à cette [adresse](https://gitlab.com/piste_lab/oauth_connectors/-/blob/master/Python/Oauth2ClientCredentialsTest.py?ref_type=heads). Il est tout de même nécessaire d'installer les modules du code suivant.

In [24]:
%pip install python-dotenv
%pip install requests-oauthlib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Pour pour envoyer des requêtes à l'API nous devons d'abord obtenir un token, après s'être inscrit sur le [site de PISTE](https://piste.gouv.fr/). Ce token est obtenu en envoyant une requête contenant mon identifiant client et mon code de client au site d'autorisation. Nous avons ensuite une autorisation d'une heure avec ce token pour exploiter l'API de Légifrance. 

In [25]:
TOKEN_URL = "https://oauth.piste.gouv.fr/api/oauth/token"

import os
from dotenv import load_dotenv
import requests
from requests_oauthlib import OAuth2Session

# Cette partie sert à récupérer les ID de connexions qui sont dans un fichier .env que l'on doit créer à chaque fois que l'on copie le repository github, cela permet de les conserver secret
load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Requette d'un token pour avoir accès à l'API 
res = requests.post(
  TOKEN_URL,
  data={
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret,
    "scope": "openid"
  }
)

# Récupération du token 
token = res.json()
client = OAuth2Session(client_id, token=token)

Maintenant en utilisant le lien d'exploitation de l'API auquel on rajoute l'endpoints qui permet d'accéder à ce que l'on veut faire nous récupérons les données. La liste des endpoints pour l'API de Légifrance est disponible [ici](https://piste.gouv.fr/index.php?option=com_apiportal&view=apitester&usage=api&apitab=tests&apiName=L%C3%A9gifrance&apiId=7e5a0e1d-ffcc-40be-a405-a1a5c1afe950&managerId=3&type=rest&apiVersion=2.4.2&Itemid=179&swaggerVersion=2.0&lang=fr).

In [26]:
API_HOST = "https://api.piste.gouv.fr/dila/legifrance/lf-engine-app"

In [45]:
import requests
import pandas as pd
import math



code_api = {
  "recherche": {
    "filtres": [
      {
        "dates": {
          "start": "1996-01-01",
          "end": "2022-08-31"
        },
        "facette": "DATE_SIGNATURE"
      }
    ],
    "sort": "SIGNATURE_DATE_DESC",
    "fromAdvancedRecherche": True,
    "champs": [
      {
        "criteres": [
          {
            "valeur": "délinquance",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "crime",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "délit",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Homicides",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Vols",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Stupéfiants",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Escroquerie",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Contrefaçon",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Sequestrations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Recels",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Proxénétisme",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Menaces",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Cambriolages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "infraction",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Attentats",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "dégradations",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          },
          {
            "valeur": "Outrages",
            "operateur": "OU",
            "typeRecherche": "UN_DES_MOTS"
          }
        ],
        "operateur": "OU",
        "typeChamp": "ALL"
      }
    ],
    "pageSize": 100,
    "pageNumber": 1,
    "operateur": "ET",
    "typePagination": "DEFAUT"
  },
  "fond":"LODA_DATE"
}




def collect_all_results(client, api_host, code):
    # Récupérer le total de résultats et calculer le nombre de pages
    response = client.post(api_host, json=code).json()
    total_results = response.get("totalResultNumber", 0)
    page_size = code["recherche"]["pageSize"]
    total_pages = math.ceil(total_results / page_size)

    print(f"Total de résultats : {total_results}")
    print(f"Nombre de pages à récupérer : {total_pages}")

    # Liste pour stocker tous les résultats
    all_results = []
    for page_number in range(1, total_pages + 1):
        print(f"Récupération de la page {page_number}/{total_pages}...")
        code["recherche"]["pageNumber"] = page_number
        response = client.post(api_host, json=code).json()
        all_results.extend(response.get("results", []))

    return all_results
    


In [46]:
api_hots = API_HOST+"/search"
client.post(api_hots, json = code_api).json()

{'executionTime': 4631,
 'results': [{'titles': [{'id': 'LEGITEXT000046244600_21-08-2023',
     'cid': 'JORFTEXT000046242448',
     'title': "Arrêté du 31 août 2022 fixant les niveaux de prise en charge des contrats d'apprentissage",
     'legalStatus': None,
     'startDate': None,
     'endDate': None,
     'nature': None}],
   'text': None,
   'type': '_doc',
   'nature': 'ARRETE',
   'origin': 'LEGI',
   'etat': 'VIGUEUR',
   'date': '2022-09-01T00:00:00.000+0000',
   'sections': [{'id': 'JORFSCTA000046242456',
     'title': 'Annexes',
     'dateVersion': '2023-08-21T00:00:00.000+0000',
     'legalStatus': 'VIGUEUR',
     'extracts': [{'id': 'LEGIARTI000047983138',
       'title': 'Annexe III',
       'legalStatus': 'VIGUEUR',
       'dateVersion': '2023-08-21T00:00:00.000+0000',
       'dateDebut': '2023-08-21T00:00:00.000+0000',
       'dateFin': '2999-01-01T00:00:00.000+0000',
       'searchFieldName': None,
       'num': 'Annexe III',
       'values': ["[...] perfectionnement s

In [47]:
all_results = collect_all_results(client, API_HOST+"/search", code_api)
def results_to_dataframe(results):
    data = []
    for result in results:
        title = result.get("titles", [{}])[0].get("title") if result.get("titles") else None
        date = result.get("date")
        id = result.get("titles", [{}])[0].get("id")
        data.append({"Titre": title, "Date": date, "ID": id})
    return pd.DataFrame(data)

df = results_to_dataframe(all_results)

Total de résultats : 14872
Nombre de pages à récupérer : 149
Récupération de la page 1/149...
Récupération de la page 2/149...
Récupération de la page 3/149...
Récupération de la page 4/149...
Récupération de la page 5/149...
Récupération de la page 6/149...
Récupération de la page 7/149...
Récupération de la page 8/149...
Récupération de la page 9/149...
Récupération de la page 10/149...
Récupération de la page 11/149...
Récupération de la page 12/149...
Récupération de la page 13/149...
Récupération de la page 14/149...
Récupération de la page 15/149...
Récupération de la page 16/149...
Récupération de la page 17/149...
Récupération de la page 18/149...
Récupération de la page 19/149...
Récupération de la page 20/149...
Récupération de la page 21/149...
Récupération de la page 22/149...
Récupération de la page 23/149...
Récupération de la page 24/149...
Récupération de la page 25/149...
Récupération de la page 26/149...
Récupération de la page 27/149...
Récupération de la page 28/149

In [48]:
df = df.drop_duplicates()
df.to_csv("resultats_legifrance.csv", index=False)